In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from bs4 import BeautifulSoup
import requests
import time, os, io

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [ ]:
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException

In [ ]:
import sys
sys.setrecursionlimit(100000)

In [ ]:
chromedriver = "/Users/Daria/Downloads/chromedriver_4" # path to the chromedriver
os.environ["webdriver.chrome.driver"] = chromedriver

**Bottoms**

In [ ]:
###URLs of bottoms/jeans for scraping

#The Cheeky Straight Jean:
url1 = 'https://www.everlane.com/products/womens-high-rise-straight-jean-regular-midblue2?collection=womens-jeans'

#Authentic Stretch High-Rise Skinny
url2 = 'https://www.everlane.com/products/womens-auth-strch-hr-skny-jean-white?collection=womens-jeans'

#The Straight Leg Crop
url3 = 'https://www.everlane.com/products/womens-straight-leg-crop-bone?collection=womens-bestsellersv2'

#The Super-Soft Straight Leg Jean
url4 = 'https://www.everlane.com/products/womens-summer-jean-distressed-indigo?collection=womens-jeans'

#The Kick Crop Jean
url5 = 'https://www.everlane.com/products/womens-kick-crop-jean-black?collection=womens-jeans'

#The Cheeky Bootcut Jean
url6 = 'https://www.everlane.com/products/womens-cheeky-bootcut-classicbluewash?collection=womens-jeans'

#The Curvy Authentic Stretch High-Rise Skinny Jean
url7 = 'https://www.everlane.com/products/womens-curvy-as-highrise-skinny-jean-midblue?collection=womens-jeans'

#The Modern Flare Jean
url8 = 'https://www.everlane.com/products/womens-modern-flare-jean-black?collection=womens-jeans'

In [ ]:
list_urls_bottoms = [url1, url2, url3, url4, url5, url6, url7, url8]

In [ ]:
# Defining function for scraping information regarding reviews from list of links
## this info will be presented as list of pages that will be parsed later

def get_pages(list_urls):
    lis_pages = []
    for link in list_urls:
        lis = []
        driver = webdriver.Chrome(chromedriver)
        driver.get(link)
        time.sleep(2)
        #to remove pop-up ad to continue 
        close_button = driver.find_element_by_xpath('//span[contains(@class, "product-login-modal__close-button")]')
        close_button.click()
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        #name of product
        name_product = soup.find('h1', class_="product-heading__name")
        lis += name_product
        try:
            #price
            price = soup.find('span', class_="product-heading__price-value")
            lis += price
        except TypeError:
            price = soup.find('span', class_="product-heading__cwyp-price")
            lis += price
        try:
            reviews = soup.find_all('div', class_="review review-index__review")
            lis += reviews
            #number of pages with review
            num = int(soup.find('div', class_='pagination-controls').find_all(class_="pagination-controls__page")[:-1][-1].text)-1
            if num > 300:
                num = 300
            for i in range(num):
                try:
                    search_button = driver.find_element_by_xpath('//button[contains(@class, "pagination-controls__page pagination-controls__page--next")]')
                    search_button.click()
                    #waiting 3 sec for loading next page review
                    time.sleep(3) 
                    soup = BeautifulSoup(driver.page_source, 'html.parser')
                    reviews = soup.find_all('div', class_="review review-index__review") 
                    lis += reviews
                except ElementClickInterceptedException:
                    continue
            driver.quit()
        except (SyntaxError, AttributeError):
            driver.quit()
            continue
        lis_pages.append(lis)
        
    return lis_pages


In [ ]:
# Making dictionary from list of pages with reviews

def get_dict(list_pages_all):
    dict_info = {}
    for j,list_pages in enumerate(list_pages_all):
        #product name
        product_name = list_pages[0].text
        #price
        price = list_pages[1]
        for i, review in enumerate(list_pages[2:]):
            #review title
            review_title = review.find(class_="review__title").text
            #the review body
            review_body = review.find(class_="review__body").text
            #Date
            date_posted = review.find('span', class_="review__attribution").text.split(',',1)[1].strip()
            #Name
            name = review.find('span', class_="review__attribution").text.split(',',1)[0].strip()
            rating = str(review.find(class_="review__rating"))[35:39]

            # Some of the information on the Website regarding Size, Height, Weight is missing
            ## Creating exceptions (Try - Ecxept) for each of these categories for handling some errors

            try:
            #Usual Size
                usual_size = review.find_all('span', class_="review__info-attribute")[0].text
                if "Usual" in usual_size:
                    pass
                else:
                    usual_size = np.nan
            except IndexError:
                usual_size = np.nan
            try:
                #Size purchased
                size_purchased = review.find_all('span', class_="review__info-attribute")[1].text
                if "Purchased" in size_purchased:
                    pass
                else:
                    size_purchased = np.nan
            except IndexError:
                size_purchased = np.nan
            try:    
                #Height
                height = review.find_all('span', class_="review__info-attribute")[2].text

            except IndexError:
                height = np.nan
            try:
            #Weight
                weight = review.find_all('span', class_="review__info-attribute")[3].text
            except IndexError:
                weight = np.nan
            try:
                #Bust size, if I were scraping tops and tees
                bust = review.find_all('span', class_="review__info-attribute")[4].text
            except IndexError:
                bust = np.nan
            dict_info[j,i] = ([name, review_title, review_body, rating, date_posted, usual_size, size_purchased, height, weight,
                             bust, product_name, price])
    return dict_info

In [ ]:
# Making DataFrame out of dictionary, correcting some info

def dict_into_df(list_pages):
    df = pd.DataFrame(list_pages).T
    df = df.reset_index()
    df.drop(['level_0', 'level_1'], axis=1, inplace=True)
    df.columns = ['name', 'review_title', 'review_body', 'rating', 'date_posted', 'usual_size', 'size_purchased', 'height', 'weight',
                         'bust', 'product_name', 'price']
    
    #correcting indexes
    
    ##1. Correcting height by moving bust
    indexes = list(df.height[df['weight'].str.contains("Height", na = False)].index)

    #moving weight from height column to the weight column
    for idx in indexes:
        df.loc[idx]['height'] = df.loc[idx]['weight']

    #setting 0 for height column where weight was previously set
    for idx in indexes:
        df.loc[idx]['weight'] = np.nan
        
    ##2. Correcting weight by moving height
    indexes = list(df.weight[df['height'].str.contains("Weight", na = False)].index)

    #moving weight from height column to the weight column
    for idx in indexes:
        df.loc[idx]['weight'] = df.loc[idx]['height']

    #setting 0 for height column where weight was previously set
    for idx in indexes:
        df.loc[idx]['height'] = np.nan
    
    ##3. Correcting bust by moving weight into right column
    indexes = list(df.weight[df['bust'].str.contains("Weight", na = False)].index)

    #moving weight from height column to the weight column
    for idx in indexes:
        df.loc[idx]['weight'] = df.loc[idx]['bust']

    #setting 0 for height column where weight was previously set
    for idx in indexes:
        df.loc[idx]['bust'] = np.nan
        
    ##4. Correcting weight by moving height
    indexes = list(df.bust[df['height'].str.contains("Bust", na = False)].index)

    #moving weight from height column to the weight column
    for idx in indexes:
        df.loc[idx]['bust'] = df.loc[idx]['height']

    #setting 0 for height column where weight was previously set
    for idx in indexes:
        df.loc[idx]['height'] = np.nan
    
    ##5. Correcting weight by moving height
    indexes = list(df.height[df['height'].str.contains("Fit", na = False)].index)


    #setting 0 for height column where weight was previously set
    for idx in indexes:
        df.loc[idx]['height'] = np.nan
    
    ##6. Correcting weight by moving height
    indexes = list(df.bust[df['weight'].str.contains("Bust", na = False)].index)

    #moving weight from height column to the weight column
    for idx in indexes:
        df.loc[idx]['bust'] = df.loc[idx]['weight']

    #setting 0 for height column where weight was previously set
    for idx in indexes:
        df.loc[idx]['weight'] = np.nan
    
    return df

In [ ]:
# getting list of pages

lis_pages_bottoms = get_pages(list_urls_bottoms)

In [ ]:
# making dictionary

bottoms_dict = get_dict(lis_pages_bottoms)

In [ ]:
# making DataFrame

df_bottoms = dict_into_df(bottoms_dict)

In [ ]:
#pickle!

df_bottoms.to_pickle("./df_bottoms.pkl")

##df_bottoms = pd.read_pickle("./df_bottoms.pkl")

**Shoes**

In [ ]:
## List of url for shoes

# The Day Glove
url01 = 'https://www.everlane.com/products/womens-day-glove-mocha?collection=womens-shoes'

# The Day Heel
url02 = 'https://www.everlane.com/products/womens-day-heel-burnt-light-taupe?collection=womens-shoes'

# The Editor Slingback
url03 = 'https://www.everlane.com/products/womens-editor-slingback-rosewood-suede?collection=womens-sale'

# The Trainer
url04 = 'https://www.everlane.com/products/womens-trainer-grey?collection=womens-sale'

# The Square Toe Chelsea Boot
url05 = 'https://www.everlane.com/products/womens-square-toe-chelsea-boot-biscuit?collection=womens-shoes'

# The Modern Babo
url06 = 'https://www.everlane.com/products/womens-modern-babo-black?collection=womens-shoes'

# The Day Loafer
url07 = 'https://www.everlane.com/products/womens-day-loafer-merlot?collection=womens-shoes'

# The Editor Heel
url08 = 'https://www.everlane.com/products/womens-editor-heel-navy-suede?collection=womens-sale'

In [ ]:
list_urls_shoes = [url01, url02, url03, url04, url05, url06, url07, url08]

In [ ]:
# getting list of pages

lis_pages_shoes = get_pages(list_urls_shoes)

In [ ]:
# making dictionary

shoes_dict = get_dict(lis_pages_shoes)

In [ ]:
# making DataFrame

df_shoes = dict_into_df(shoes_dict)

In [ ]:
#pickle!

df_shoes.to_pickle("./df_shoes.pkl")

##df_shoes = pd.read_pickle("./df_shoes.pkl")

**_Disclaimer:_** <br>
The scraping process that I was doing during the Project was performed in another jupyter notebook. 
This notebook represents a cleaner version as if I were scraping once again.<br>
(Since scraping is time consuming I decided not to re-scrape all over again, but show code and process)<br>
Also, the way of scraping provided in this notebook may vary with time, for example, if the website has changed (regarding pop-up windows and etc.)<br>
This code worked as of late May 2020.